In [1]:
import os
import cv2
import numpy as np
from skimage import metrics

# Caminhos para as pastas de entrada (imagens) e saída (resultados)
input_folder = 'ultrassound\\cropped'
mask_folder = 'ultrassound\\masks'
output_folder = 'result3'

# Obter o caminho completo para as pastas
input_folder_full = os.path.join(os.getcwd(), input_folder)
mask_folder_full = os.path.join(os.getcwd(), mask_folder)
output_folder_full = os.path.join(os.getcwd(), output_folder)

# Verificar se as pastas existem
if not os.path.exists(input_folder_full) or not os.path.exists(mask_folder_full):
    print(f"As pastas de entrada '{input_folder_full}' ou '{mask_folder_full}' não foram encontradas.")
    exit()

# Criar a pasta de saída se não existir
os.makedirs(output_folder_full, exist_ok=True)

# Lista de arquivos na pasta de entrada
image_files = [f for f in os.listdir(input_folder_full) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

# Inicializar variáveis para métricas agregadas
total_intersection = 0
total_union = 0

# Loop sobre todas as imagens na pasta de entrada
for image_file in image_files:
    # Construir o caminho completo da imagem de entrada
    image_path = os.path.join(input_folder_full, image_file)

    # Construir o caminho completo da máscara de referência
    mask_path = os.path.join(mask_folder_full, image_file)

    # Tente carregar a imagem e a máscara
    image = cv2.imread(image_path)
    mask_reference = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Verifique se a imagem e a máscara foram carregadas com sucesso
    if image is None or mask_reference is None:
        print(f'Erro ao carregar a imagem ou a máscara de referência: {image_path}')
        continue

    # Aplicar a segmentação (exemplo: binarização de Otsu)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, segmented_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Calcular a interseção e união para métricas agregadas
    total_intersection += np.logical_and(mask_reference.flatten(), segmented_mask.flatten()).sum()
    total_union += np.logical_or(mask_reference.flatten(), segmented_mask.flatten()).sum()

    # Salvar a máscara segmentada na pasta de saída
    output_mask_path = os.path.join(output_folder_full, f'segmented_{image_file}')
    cv2.imwrite(output_mask_path, segmented_mask)

# Calcular as métricas agregadas
dice_coefficient_aggregated = (2 * total_intersection) / (total_intersection + total_union)
jaccard_index_aggregated = total_intersection / total_union

# Imprimir os resultados agregados
print('Métricas Agregadas:')
print(f'Dice Coefficient Agregado: {dice_coefficient_aggregated}')
print(f'Jaccard Index Agregado (IoU): {jaccard_index_aggregated}')


Métricas Agregadas:
Dice Coefficient Agregado: 0.5923242798771723
Jaccard Index Agregado (IoU): 0.42078176913180587


### No código fornecido, a segmentação foi realizada utilizando a binarização de Otsu. Esta é uma técnica simples de segmentação baseada em limiar, onde o limiar é automaticamente escolhido usando o método de Otsu para otimizar a separação entre os pixels de primeiro e segundo plano.

### As métricas aplicadas foram o Dice Coefficient e o Jaccard Index (IoU). Aqui está um resumo de como essas métricas foram calculadas no código:

### Dice Coefficient:

### Foi calculado usando a fórmula: 
2
×
intersection
sum of pixels in mask_reference
+
sum of pixels in segmented_mask
sum of pixels in mask_reference+sum of pixels in segmented_mask
2×intersection
​
 
Essa métrica mede a sobreposição entre a máscara segmentada e a máscara de referência.
Jaccard Index (IoU):

Foi calculado usando a fórmula: 
intersection
union
union
intersection
​
 
Esta métrica também mede a sobreposição entre a máscara segmentada e a máscara de referência.
As máscaras de referência foram obtidas a partir da pasta masks na base de dados, e a máscara segmentada foi gerada usando a binarização de Otsu. Em seguida, as métricas foram calculadas para avaliar a qualidade da segmentação em comparação com a máscara de referência.